# Ratebeer Text Feature Engineering


In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import fasttext
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf, Reader, dump
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os

In [2]:
df = pd.read_pickle('./train_set.pkl')

In [3]:
test_df = pd.read_pickle('./test_set.pkl')

In [3]:
df.head()

,beerID,beer_name,abv,abv_listed,brewerID,beer_style,reviewer_username,review_appearance,review_aroma,review_palate,...,vader_pos,time_from_beginning,weekend_review,time_from_beginning_lookup,weekday_central_lookup,month_lookup,distance_from_july_lookup,vader_neg_lookup,vader_neu_lookup,vader_pos_lookup
367332,5868,St Peters Organic Ale,4.5,True,974,Bitter,abemorsten,3,6,4,...,0.179,300412800,True,"[5868, abemorsten, 300412800]","[5868, abemorsten, Sunday]","[5868, abemorsten, 10]","[5868, abemorsten, 3]","[5868, abemorsten, 0.064]","[5868, abemorsten, 0.757]","[5868, abemorsten, 0.179]"
1841875,493,Busch Light,4.1,True,84,Pale Lager,beerdrinker101,1,1,1,...,0.100,186105600,True,"[493, beerdrinker101, 186105600]","[493, beerdrinker101, Sunday]","[493, beerdrinker101, 03]","[493, beerdrinker101, 4]","[493, beerdrinker101, 0.0]","[493, beerdrinker101, 0.9]","[493, beerdrinker101, 0.1]"
1420827,85344,Auburn Alehouse Red 45,6.0,True,8572,Irish Ale,bb,4,7,3,...,0.000,250473600,False,"[85344, bb, 250473600]","[85344, bb, Wednesday]","[85344, bb, 03]","[85344, bb, 4]","[85344, bb, 0.0]","[85344, bb, 1.0]","[85344, bb, 0.0]"
1709329,1763,Greene King Abbot Ale &#40;Filtered&#41;,5.0,True,143,Premium Bitter/ESB,jazz88,4,5,3,...,0.107,94867200,False,"[1763, jazz88, 94867200]","[1763, jazz88, Monday]","[1763, jazz88, 04]","[1763, jazz88, 3]","[1763, jazz88, 0.128]","[1763, jazz88, 0.764]","[1763, jazz88, 0.107]"
327581,2396,Atlantic Coal Porter,5.8,True,253,Porter,zach8270,3,6,4,...,0.000,141004800,False,"[2396, zach8270, 141004800]","[2396, zach8270, Wednesday]","[2396, zach8270, 09]","[2396, zach8270, 2]","[2396, zach8270, 0.062]","[2396, zach8270, 0.938]","[2396, zach8270, 0.0]"


In [4]:
test_df.head()

,beerID,beer_name,abv,abv_listed,brewerID,beer_style,reviewer_username,review_appearance,review_aroma,review_palate,review_taste,review_overall,review_text,review_unix_time,weekday_central,day_of_year,month,distance_from_july
2081618,136550,Lost Abbey Sinners 10,8.0,True,7043,Sour Ale/Wild Ale,MrChopin,2,5,2,6,9,Bottle opens with the sound of repressed fart...,1306713600,Sunday,149,05,2
1729409,23373,Brewbakers Surf City Blonde Ale,6.0,False,390,Golden Ale/Blond Ale,bartalone,3,3,2,5,7,"Not a strong flavor mild, very very mild mu...",1306713600,Sunday,149,05,2
2281210,48725,Port Washington Hop-2-It IPA,6.0,True,1336,India Pale Ale &#40;IPA&#41;,nuplastikk,3,7,3,7,14,Sample at Hops Haven. Hazy orange tan. Thin...,1306713600,Sunday,149,05,2
1122846,101199,Founders Cerise,6.5,True,554,Fruit Beer,hoosiers1994,4,6,3,6,11,12 oz bottle poured into a tulip. Aroma of b...,1306713600,Sunday,149,05,2
1238760,145953,Muirhouse Magnum Mild,4.5,True,11110,Mild Ale,maeib,4,8,3,7,15,Cask conditioned\tgravity dispense\tNorthants...,1306713600,Sunday,149,05,2


In [7]:
stopWords = set(stopwords.words('english'))

In [8]:
df['text_filtered'] = df['review_text']\
.apply( lambda x: " ".join([i for i in x.lower().split() if i not in stopWords]) )

In [9]:
test_df['text_filtered'] = test_df['review_text']\
.apply( lambda x: " ".join([i for i in x.lower().split() if i not in stopWords]) )

In [10]:
# now remove punctuation
df["text_simple"] = df['text_filtered'].str.replace('[^\w\s]','')

In [11]:
test_df["text_simple"] = test_df['text_filtered'].str.replace('[^\w\s]','')

In [3]:

#sentiment_scorer = SentimentIntensityAnalyzer()
#df['sentiment_vader'] = df['review_text'].apply(lambda x: sentiment_scorer.polarity_scores(x))
#df1, df2 = np.array_split(df, 2)
#df1.to_pickle(path='./post_vader1.pkl')
#df2.to_pickle(path='./post_vader2.pkl')

df1 = pd.read_pickle('post_vader1.pkl')
df2 = pd.read_pickle('post_vader2.pkl')
df=pd.concat([df1,df2])

In [12]:
#test_df.to_pickle('test_postVader.pkl')
test_df=pd.read_pickle('test_postVader.pkl')

In [4]:
df.columns.values

array(['beerID', 'beer_name', 'abv', 'abv_listed', 'brewerID',
       'beer_style', 'reviewer_username', 'review_appearance',
       'review_aroma', 'review_palate', 'review_taste', 'review_overall',
       'review_text', 'review_unix_time', 'weekday_central', 'day_of_year',
       'month', 'distance_from_july', 'text_filtered', 'text_simple',
       'sentiment_vader'], dtype=object)

In [5]:
df.head()

,beerID,beer_name,abv,abv_listed,brewerID,beer_style,reviewer_username,review_appearance,review_aroma,review_palate,...,review_overall,review_text,review_unix_time,weekday_central,day_of_year,month,distance_from_july,text_filtered,text_simple,sentiment_vader
2291588,121,Stone City Hefeweizen,6.0,False,23,German Hefeweizen,lazarus99,5,8,4,...,20,I love this kind of beer. It seems to be the ...,955497600,Tuesday,102,04,3,love kind beer. seems thing apart corona type ...,love kind beer seems thing apart corona type b...,"{'neu': 0.819, 'compound': 0.6369, 'pos': 0.18..."
1651460,132,Newcastle Brown Ale,4.7,True,751,Brown Ale,billb,5,6,5,...,17,This has to be one of my favorite drinking be...,955497600,Tuesday,102,04,3,"one favorite drinking beers. light, dark. good...",one favorite drinking beers light dark good cl...,"{'neu': 0.798, 'compound': 0.8225, 'pos': 0.20..."
2146614,286,Worthington White Shield,5.6,True,12871,Premium Bitter/ESB,wade,3,10,4,...,18,Excellent ale with a strong aroma and initial...,955843200,Saturday,106,04,3,excellent ale strong aroma initial flavor. aft...,excellent ale strong aroma initial flavor afte...,"{'neu': 0.568, 'compound': 0.81, 'pos': 0.432,..."
2182000,544,Saranac Pale Ale,5.5,True,92,English Pale Ale,billb,4,7,5,...,17,Pretty tasty for a pale ale. This is one of ...,956448000,Saturday,113,04,3,pretty tasty pale ale. one faves saranac. nice...,pretty tasty pale ale one faves saranac nice h...,"{'neu': 0.691, 'compound': 0.8807, 'pos': 0.30..."
2171356,547,Saranac Black and Tan,5.1,True,92,Stout,billb,3,7,2,...,10,I a big fan of Black and Tans but this one is...,956448000,Saturday,113,04,3,big fan black tans one rather disappointing. a...,big fan black tans one rather disappointing ar...,"{'neu': 0.701, 'compound': 0.0516, 'pos': 0.16..."


In [13]:
# split vader sentiment scores into seperate columns
df['vader_neg']=df['sentiment_vader'].apply(lambda x: float(x['neg']))
df['vader_neu']=df['sentiment_vader'].apply(lambda x: float(x['neu']))
df['vader_pos']=df['sentiment_vader'].apply(lambda x: float(x['pos']))

Create a feature listing number of seconds since first review on record. This might be easier to train on than the unix time value.

In [7]:
df['review_unix_time'].min()

955497600

In [15]:
test_df['review_unix_time'].min()

1306713600

In [16]:
df['time_from_beginning']=df['review_unix_time']-955497600

In [17]:
test_df['time_from_beginning']=test_df['review_unix_time']-955497600

In [18]:
# create boolean feature saying if review is on weekend
df['weekend_review']=df['weekday_central'].apply(lambda x: True if x in ['Friday', 'Saturday', 'Sunday'] else False)

In [19]:
# create boolean feature saying if review is on weekend
test_df['weekend_review']=test_df['weekday_central'].apply(lambda x: True if x in ['Friday', 'Saturday', 'Sunday'] else False)

In [10]:
# create this list of lists so we can aggregate into a dict later 
# This approach is stupid! Ended up running out of memory or it became too computationally expensive
"""df['text_lookup'] = df[['beerID','reviewer_username', 'text_simple']].values.tolist()
df['abv_lookup']=df[['beerID', 'reviewer_username', 'abv']].values.tolist()
df['brewerID_lookup']=df[['beerID','reviewer_username','brewerID']].values.tolist()
df['beerStyle_lookup']=df[['beerID','reviewer_username','beer_style']].values.tolist()
df['review_appearance_lookup']=df[['beerID','reviewer_username','review_appearance']].values.tolist()
df['review_aroma_lookup']=df[['beerID', 'reviewer_username', 'review_aroma']].values.tolist()
df['review_palate_lookup']=df[['beerID', 'reviewer_username', 'review_palate']].values.tolist()
df['review_taste_lookup']=df[['beerID', 'reviewer_username', 'review_taste']].values.tolist()
df['review_overall_lookup']=df[['beerID', 'reviewer_username', 'review_overall']].values.tolist()
df['time_from_beginning_lookup']=df[['beerID', 'reviewer_username', 'time_from_beginning']].values.tolist()
df['weekday_central_lookup']=df[['beerID', 'reviewer_username', 'weekday_central']].values.tolist()
df['month_lookup']=df[['beerID', 'reviewer_username', 'month']].values.tolist()
df['distance_from_july_lookup']=df[['beerID', 'reviewer_username', 'distance_from_july']].values.tolist()
df['vader_neg_lookup']=df[['beerID', 'reviewer_username', 'vader_neg']].values.tolist()
df['vader_neu_lookup']=df[['beerID', 'reviewer_username', 'vader_neu']].values.tolist()
df['vader_pos_lookup']=df[['beerID', 'reviewer_username', 'vader_pos']].values.tolist()"""


"df['text_lookup'] = df[['beerID','reviewer_username', 'text_simple']].values.tolist()\ndf['abv_lookup']=df[['beerID', 'reviewer_username', 'abv']].values.tolist()\ndf['brewerID_lookup']=df[['beerID','reviewer_username','brewerID']].values.tolist()\ndf['beerStyle_lookup']=df[['beerID','reviewer_username','beer_style']].values.tolist()\ndf['review_appearance_lookup']=df[['beerID','reviewer_username','review_appearance']].values.tolist()\ndf['review_aroma_lookup']=df[['beerID', 'reviewer_username', 'review_aroma']].values.tolist()\ndf['review_palate_lookup']=df[['beerID', 'reviewer_username', 'review_palate']].values.tolist()\ndf['review_taste_lookup']=df[['beerID', 'reviewer_username', 'review_taste']].values.tolist()\ndf['review_overall_lookup']=df[['beerID', 'reviewer_username', 'review_overall']].values.tolist()\ndf['time_from_beginning_lookup']=df[['beerID', 'reviewer_username', 'time_from_beginning']].values.tolist()\ndf['weekday_central_lookup']=df[['beerID', 'reviewer_username', 

In [11]:
df.columns.values

array(['beerID', 'beer_name', 'abv', 'abv_listed', 'brewerID',
       'beer_style', 'reviewer_username', 'review_appearance',
       'review_aroma', 'review_palate', 'review_taste', 'review_overall',
       'review_text', 'review_unix_time', 'weekday_central', 'day_of_year',
       'month', 'distance_from_july', 'text_filtered', 'text_simple',
       'sentiment_vader', 'vader_neg', 'vader_neu', 'vader_pos',
       'time_from_beginning', 'weekend_review'], dtype=object)

## Aggregate features

### Aggregate by User

In [12]:
# below old version of aggregation before I realised I would need to keep tab of which review corresponds to which 
# value so I could adjust the aggregate values by removing the values corresponding to a specific item/reviewer
# when we join later
"""aggregation={
    'beerID' : {
        'beerID_count' : 'count'
    },
    'abv' : {
        'avgABV' : 'mean',
        'medianABV' : 'median',
        'maxABV' : 'max',
        'minABV' : 'min',
        'ABVrange' : lambda x: max(x) - min(x)
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: int(sum(x)) 
    },
    'brewerID' : {
        'num_brewers' : lambda x: x.nunique()
    },
    'beer_style' : {
        'numBeerStyles' : lambda x : x.nunique()
    },
    'review_appearance' : {
        'avgAppearanceScore' : 'mean',
        'medianAppearanceScore' : 'median',
        'maxAppearanceScore' : 'max',
        'minAppearanceScore' : 'min',
        'appearanceScoreRange' : lambda x: max(x) - min(x)
    },
    'review_aroma' : {
        'avgAromaScore' : 'mean',
        'medianAromaScore' : 'median',
        'maxAromaScore' : 'max',
        'minAromaScore' : 'min',
        'aromaScoreRange' : lambda x: max(x) - min(x)
    },
    'review_palate' : {
        'avgPalateScore' : 'mean',
        'medianPalateScore' : 'median',
        'maxPalateScore' : 'max',
        'minPalateScore' : 'min',
        'palateScoreRange' : lambda x: max(x) - min(x)
    },
    'review_taste' : {
        'avgTasteScore' : 'mean',
        'medianTasteScore' : 'median',
        'maxTasteScore' : 'max',
        'minTasteScore' : 'min',
        'tasteScoreRange' : lambda x: max(x) - min(x)
    },
    'review_overall' : {
        'avgOverallScore' : 'mean',
        'medianOverallScore' : 'median',
        'maxOverallScore' : 'max',
        'minOverallScore' : 'min',
        'overallScoreRange' : lambda x: max(x) - min(x)
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'time_from_beginning' : {
        'avgTimestamp' : 'mean',
        'medianTimestamp' : 'median',
        'maxTimestamp' : 'max',
        'minTimestamp' : 'min',
        'TimestampRange' : lambda x: max(x) - min(x)
    },
    'weekday_central' : {
        'most_common_weekday' : lambda x: x.value_counts().index[0],
        'number_of_weekdays' : lambda x: x.nunique()
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month' : {
        'most_common_month' : lambda x: x.value_counts().index[0],
        'number_of_months' : lambda x: x.nunique()
    },
    'distance_from_july' : {
        'avgMonthsFromJuly' : 'mean',
        'medianMonthsFromJuly' : 'median',
        'maxMonthsFromJuly' : 'max',
        'minMonthsFromJuly' : 'min',
        'monthsFromJulyRange' : lambda x: max(x) - min(x)
    },
    'vader_neg' : {
        'avgVaderNeg' : 'mean',
        'medianVaderNeg' : 'median',
        'maxVaderNeg' : 'max',
        'minVaderNeg' : 'min',
        'vaderNegRange' : lambda x: max(x) - min(x)
    },
    'vader_neu' : {
        'avgVaderNeu' : 'mean',
        'medianVaderNeu' : 'median',
        'maxVaderNeu' : 'max',
        'minVaderNeu' : 'min',
        'vaderNeuRange' : lambda x: max(x) - min(x)
    },
    'vader_pos' : {
        'avgVaderPos' : 'mean',
        'medianVaderPos' : 'median',
        'maxVaderPos' : 'max',
        'minVaderPos' : 'min',
        'vaderPosRange' : lambda x: max(x) - min(x)
    }
}
aggByUser_df = df.groupby('reviewer_username').agg(aggregation)"""

"aggregation={\n    'beerID' : {\n        'beerID_count' : 'count'\n    },\n    'abv' : {\n        'avgABV' : 'mean',\n        'medianABV' : 'median',\n        'maxABV' : 'max',\n        'minABV' : 'min',\n        'ABVrange' : lambda x: max(x) - min(x)\n    },\n    'abv_listed' : {\n        'num_abv_listed' : lambda x: int(sum(x)) \n    },\n    'brewerID' : {\n        'num_brewers' : lambda x: x.nunique()\n    },\n    'beer_style' : {\n        'numBeerStyles' : lambda x : x.nunique()\n    },\n    'review_appearance' : {\n        'avgAppearanceScore' : 'mean',\n        'medianAppearanceScore' : 'median',\n        'maxAppearanceScore' : 'max',\n        'minAppearanceScore' : 'min',\n        'appearanceScoreRange' : lambda x: max(x) - min(x)\n    },\n    'review_aroma' : {\n        'avgAromaScore' : 'mean',\n        'medianAromaScore' : 'median',\n        'maxAromaScore' : 'max',\n        'minAromaScore' : 'min',\n        'aromaScoreRange' : lambda x: max(x) - min(x)\n    },\n    'review_

In [13]:
# See above. This approach, of keeping aggregate features in lists was way too computationally expensive
"""aggregation={
    'beerID' : {
        'beerID_count' : 'count'
    },
    'abv_lookup' : {
        'abv_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: int(sum(x)) 
    },
    'brewerID_lookup' : {
        'brewerID_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'beerStyle_lookup' : {
        'beerStyle_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_appearance_lookup' : {
        'reviewAppearance_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_aroma_lookup' : {
        'reviewAroma_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_palate_lookup' : {
        'reviewPalate_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_taste_lookup' : {
        'reviewTaste_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'review_overall_lookup' : {
        'reviewOverall_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'time_from_beginning_lookup' : {
        'time_from_beginning_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'weekday_central_lookup' : {
        'weekday_central_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month_lookup' : {
        'month_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'distance_from_july_lookup' : {
        'distance_from_july_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'vader_neg_lookup' : {
        'vader_neg_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'vader_neu_lookup' : {
        'vader_neu_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'vader_pos_lookup' : {
        'vader_pos_list' : lambda x : [[item[0],item[2]] for item in x]
    }
}
aggByUser_df = df.groupby('reviewer_username').agg(aggregation)"""

"aggregation={\n    'beerID' : {\n        'beerID_count' : 'count'\n    },\n    'abv_lookup' : {\n        'abv_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'abv_listed' : {\n        'num_abv_listed' : lambda x: int(sum(x)) \n    },\n    'brewerID_lookup' : {\n        'brewerID_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'beerStyle_lookup' : {\n        'beerStyle_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_appearance_lookup' : {\n        'reviewAppearance_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_aroma_lookup' : {\n        'reviewAroma_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_palate_lookup' : {\n        'reviewPalate_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_taste_lookup' : {\n        'reviewTaste_list' : lambda x : [[item[0],item[2]] for item in x]\n    },\n    'review_overall_lookup' : {\n        'reviewOverall_list' 

In [14]:
#aggByUser_df['text_dict'] = aggByUser_df['text_lookup']['text_list'].apply(lambda x: {item[0]:item[1] for item in x})

#test=aggByUser_df['text_lookup']['text_dict'][4]
#{x[0]:x[1] for x in test}

In [15]:
#aggByUser_df.columns = aggByUser_df.columns.droplevel()

In [16]:
#aggByUser_df.head()

In [17]:
#aggByUser_df['weekend_review']

In [18]:
#aggByUser_df['beerID_count'].head()

In [19]:
#aggByUser_df['beerID_count'].mean()

In [20]:
"""aggregation2={
    'reviewer_username' : {
        'user_count' : 'count'
    },
    'abv' : {
        'ABV' : lambda x: x.value_counts().index[0]
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: x.value_counts().index[0]
    },
    'brewerID' : {
        'num_brewers' : lambda x: x.value_counts().index[0]
    },
    'beer_style' : {
        'numBeerStyles' : lambda x: x.value_counts().index[0]
    },
    'review_appearance_lookup' : {
        'reviewAppearance_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'review_aroma_lookup' : {
        'reviewAroma_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'review_palate_lookup' : {
        'reviewPalate_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'review_taste_lookup' : {
        'reviewTaste_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'review_overall_lookup' : {
        'reviewOverall_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'time_from_beginning_lookup' : {
        'time_from_beginning_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'weekday_central_lookup' : {
        'weekday_central_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month_lookup' : {
        'month_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'distance_from_july_lookup' : {
        'distance_from_july_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'vader_neg_lookup' : {
        'vader_neg_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'vader_neu_lookup' : {
        'vader_neu_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'vader_pos_lookup' : {
        'vader_pos_list' : lambda x : [[item[1],item[2]] for item in x]
    }
}
aggByProduct_df = df.groupby('beerID').agg(aggregation2)"""

"aggregation2={\n    'reviewer_username' : {\n        'user_count' : 'count'\n    },\n    'abv' : {\n        'ABV' : lambda x: x.value_counts().index[0]\n    },\n    'abv_listed' : {\n        'num_abv_listed' : lambda x: x.value_counts().index[0]\n    },\n    'brewerID' : {\n        'num_brewers' : lambda x: x.value_counts().index[0]\n    },\n    'beer_style' : {\n        'numBeerStyles' : lambda x: x.value_counts().index[0]\n    },\n    'review_appearance_lookup' : {\n        'reviewAppearance_list' : lambda x : [[item[1],item[2]] for item in x]\n    },\n    'review_aroma_lookup' : {\n        'reviewAroma_list' : lambda x : [[item[1],item[2]] for item in x]\n    },\n    'review_palate_lookup' : {\n        'reviewPalate_list' : lambda x : [[item[1],item[2]] for item in x]\n    },\n    'review_taste_lookup' : {\n        'reviewTaste_list' : lambda x : [[item[1],item[2]] for item in x]\n    },\n    'review_overall_lookup' : {\n        'reviewOverall_list' : lambda x : [[item[1],item[2]] 

In [21]:
# Thia will now have to waint until aggretation from all the list features is done
# rate of reviewing - beerID_count/timeStampRange
#aggByProduct_df['rate_of_posting'] = aggByProduct_df['reviewer_username']['user_count']/aggByProduct_df['time_from_beginning']['TimestampRange']

# weekday variety index - number_of_weekdays/beerID_count
#aggByProduct_df['weekday_variety_index'] = aggByProduct_df['weekday_central']['number_of_weekdays']/aggByProduct_df['reviewer_username']['user_count']

# month variety index - number_of_months/beerID_count
#aggByProduct_df['month_variety_index'] = aggByProduct_df['month']['number_of_months']/aggByProduct_df['reviewer_username']['user_count']

# weekend review ratio - num_weekend_reviews/beerID_count
#aggByProduct_df['weekend_review_ratio'] = aggByProduct_df['weekend_review']['num_weekend_reviews']/aggByProduct_df['reviewer_username']['user_count']



In [22]:
#aggByProduct_df.columns = aggByProduct_df.columns.droplevel()

In [23]:
#aggByProduct_df.head()

In [24]:
#aggByProduct_df['reviewer_username'].mean()

## Collaborative filtering features


In [32]:
df.columns

Index(['beerID', 'beer_name', 'abv', 'abv_listed', 'brewerID', 'beer_style',
       'reviewer_username', 'review_appearance', 'review_aroma',
       'review_palate', 'review_taste', 'review_overall', 'review_text',
       'review_unix_time', 'weekday_central', 'day_of_year', 'month',
       'distance_from_july', 'text_filtered', 'text_simple', 'sentiment_vader',
       'text_lookup', 'abv_lookup', 'brewerID_lookup', 'beerStyle_lookup',
       'review_appearance_lookup', 'review_aroma_lookup',
       'review_palate_lookup', 'review_taste_lookup', 'review_overall_lookup',
       'vader_neg', 'vader_neu', 'vader_pos', 'time_from_beginning',
       'weekend_review', 'time_from_beginning_lookup',
       'weekday_central_lookup', 'month_lookup', 'distance_from_july_lookup',
       'vader_neg_lookup', 'vader_neu_lookup', 'vader_pos_lookup'],
      dtype='object')

In [33]:
df[['review_appearance', 'review_aroma', 'review_palate', 'review_taste', 'review_overall']]

,review_appearance,review_aroma,review_palate,review_taste,review_overall
2291588,5,8,4,10,20
1651460,5,6,5,9,17
2146614,3,10,4,9,18
2182000,4,7,5,9,17
2171356,3,7,2,6,10
2696165,4,10,4,9,18
1710138,4,6,2,2,3
144972,2,1,2,2,2
2694336,4,9,5,8,17
446495,2,3,1,3,5


We must split the training set in two. Half will be used to do collaborative filtering. The other half will be used to train a classifier. This 50-50 ratio picked slightly naively. May change this in the future.

In [34]:
#SVD_set, train_set = train_test_split(df, test_size=0.5)

In [2]:
#SVD_set.to_pickle(path='SVD_set.pkl')
#train_set.to_pickle(path='train_set.pkl')
#test_df.to_pickle('testBeforeJoin.pkl')

SVD_set = pd.read_pickle('SVD_set.pkl')
trainTemp = pd.read_pickle('train_set.pkl')
test_df = pd.read_pickle('testBeforeJoin.pkl')
#NB: RERUN ALL THE COLLABORATIVE FILTERING MODELS!!!

In [3]:
# Add an colun to identify train and test rows
trainTemp['testData']=False
test_df['testData']=True

In [4]:
train_set=pd.concat([trainTemp,test_df])

In [7]:
#train_set[train_set['testData']==False].head()

In [17]:
SVD_set.count()

beerID                        1315873
beer_name                     1315873
abv                           1315873
abv_listed                    1315873
brewerID                      1315873
beer_style                    1315873
reviewer_username             1315873
review_appearance             1315873
review_aroma                  1315873
review_palate                 1315873
review_taste                  1315873
review_overall                1315873
review_text                   1315873
review_unix_time              1315873
weekday_central               1315873
day_of_year                   1315873
month                         1315873
distance_from_july            1315873
text_filtered                 1315873
text_simple                   1315873
sentiment_vader               1315873
text_lookup                   1315873
abv_lookup                    1315873
brewerID_lookup               1315873
beerStyle_lookup              1315873
review_appearance_lookup      1315873
review_aroma

In [18]:
train_set.count()

beerID                        1315873
beer_name                     1315873
abv                           1315873
abv_listed                    1315873
brewerID                      1315873
beer_style                    1315873
reviewer_username             1315873
review_appearance             1315873
review_aroma                  1315873
review_palate                 1315873
review_taste                  1315873
review_overall                1315873
review_text                   1315873
review_unix_time              1315873
weekday_central               1315873
day_of_year                   1315873
month                         1315873
distance_from_july            1315873
text_filtered                 1315873
text_simple                   1315873
sentiment_vader               1315873
text_lookup                   1315873
abv_lookup                    1315873
brewerID_lookup               1315873
beerStyle_lookup              1315873
review_appearance_lookup      1315873
review_aroma

In [3]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_appearance']], reader)
data.split(n_folds=5)

appearanaceSVD = SVD()
evaluate(appearanaceSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/appearanaceSVD_dump')
dump.dump(file_name, algo=appearanaceSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1


KeyboardInterrupt: 

In [81]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_aroma']], reader)
data.split(n_folds=5)

aromaSVD = SVD()
evaluate(aromaSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/aromaSVD_dump')
dump.dump(file_name, algo=aromaSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 1.1902
MAE:  0.8873
------------
Fold 2
RMSE: 1.1907
MAE:  0.8882
------------
Fold 3
RMSE: 1.1888
MAE:  0.8855
------------
Fold 4
RMSE: 1.1868
MAE:  0.8849
------------
Fold 5
RMSE: 1.1899
MAE:  0.8883
------------
------------
Mean RMSE: 1.1893
Mean MAE : 0.8868
------------
------------


In [79]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_palate']], reader)
data.split(n_folds=5)

palateSVD = SVD()
evaluate(palateSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/palateSVD_dump')
dump.dump(file_name, algo=palateSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.6731
MAE:  0.5179
------------
Fold 2
RMSE: 0.6732
MAE:  0.5178
------------
Fold 3
RMSE: 0.6737
MAE:  0.5175
------------
Fold 4
RMSE: 0.6738
MAE:  0.5185
------------
Fold 5
RMSE: 0.6744
MAE:  0.5189
------------
------------
Mean RMSE: 0.6736
Mean MAE : 0.5181
------------
------------


In [82]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_taste']], reader)
data.split(n_folds=5)

tasteSVD = SVD()
evaluate(tasteSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/tasteSVD_dump')
dump.dump(file_name, algo=tasteSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 1.2252
MAE:  0.9163
------------
Fold 2
RMSE: 1.2243
MAE:  0.9148
------------
Fold 3
RMSE: 1.2239
MAE:  0.9150
------------
Fold 4
RMSE: 1.2256
MAE:  0.9160
------------
Fold 5
RMSE: 1.2290
MAE:  0.9194
------------
------------
Mean RMSE: 1.2256
Mean MAE : 0.9163
------------
------------


In [83]:
reader = Reader(rating_scale=(1, 20))
data = Dataset.load_from_df(SVD_set[['reviewer_username', 'beerID', 'review_overall']], reader)
data.split(n_folds=5)

overallSVD = SVD()
evaluate(overallSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/overallSVD_dump')
dump.dump(file_name, algo=overallSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 2.4827
MAE:  1.8149
------------
Fold 2
RMSE: 2.4896
MAE:  1.8182
------------
Fold 3
RMSE: 2.4831
MAE:  1.8161
------------
Fold 4
RMSE: 2.4791
MAE:  1.8122
------------
Fold 5
RMSE: 2.4970
MAE:  1.8230
------------
------------
Mean RMSE: 2.4863
Mean MAE : 1.8169
------------
------------


In [11]:
file_name1 = os.path.expanduser('~/appearanaceSVD_dump')
_, appearanaceSVD = dump.load(file_name1)

file_name2 = os.path.expanduser('~/aromaSVD_dump')
_, aromaSVD = dump.load(file_name2)

file_name3 = os.path.expanduser('~/palateSVD_dump')
_, palateSVD = dump.load(file_name3)

file_name4 = os.path.expanduser('~/tasteSVD_dump')
_, tasteSVD = dump.load(file_name4)

file_name5 = os.path.expanduser('~/overallSVD_dump')
_, overallSVD = dump.load(file_name5)

file_name = os.path.expanduser('~/benchmarkSVD_dump')
_, standardSVD = dump.load(file_name)

In [79]:
reader = Reader(rating_scale=(1, 20))
data = Dataset.load_from_df(df[['reviewer_username', 'beerID', 'review_overall']], reader)
data.split(n_folds=5)

benchmarkSVD = SVD()
evaluate(overallSVD, data, measures=['RMSE', 'MAE'])

file_name = os.path.expanduser('~/benchmarkSVD_dump')
dump.dump(file_name, algo=benchmarkSVD)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 2.4353
MAE:  1.7737
------------
Fold 2
RMSE: 2.4324
MAE:  1.7728
------------
Fold 3
RMSE: 2.4315
MAE:  1.7704
------------
Fold 4
RMSE: 2.4329
MAE:  1.7743
------------
Fold 5
RMSE: 2.4371
MAE:  1.7771
------------
------------
Mean RMSE: 2.4338
Mean MAE : 1.7737
------------
------------


In [2]:
file_name1 = os.path.expanduser('~/appearanaceSVD_dump')
_, appearanaceSVD = dump.load(file_name1)

file_name2 = os.path.expanduser('~/aromaSVD_dump')
_, aromaSVD = dump.load(file_name2)

file_name3 = os.path.expanduser('~/palateSVD_dump')
_, palateSVD = dump.load(file_name3)

file_name4 = os.path.expanduser('~/tasteSVD_dump')
_, tasteSVD = dump.load(file_name4)

file_name5 = os.path.expanduser('~/overallSVD_dump')
_, overallSVD = dump.load(file_name5)

file_name = os.path.expanduser('~/benchmarkSVD_dump')
_, standardSVD = dump.load(file_name)

In [10]:
aggregation={
    'beerID' : {
        'beerID_count' : 'count'
    },
    'abv' : {
        'avgABV' : 'mean',
        'medianABV' : 'median',
        'maxABV' : 'max',
        'minABV' : 'min',
        'ABVrange' : lambda x: max(x) - min(x)
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: int(sum(x)) 
    },
    'brewerID' : {
        'num_brewers' : lambda x: x.nunique()
    },
    'beer_style' : {
        'numBeerStyles' : lambda x : x.nunique()
    },
    'review_appearance' : {
        'avgAppearanceScore' : 'mean',
        'medianAppearanceScore' : 'median',
        'maxAppearanceScore' : 'max',
        'minAppearanceScore' : 'min',
        'appearanceScoreRange' : lambda x: max(x) - min(x)
    },
    'review_aroma' : {
        'avgAromaScore' : 'mean',
        'medianAromaScore' : 'median',
        'maxAromaScore' : 'max',
        'minAromaScore' : 'min',
        'aromaScoreRange' : lambda x: max(x) - min(x)
    },
    'review_palate' : {
        'avgPalateScore' : 'mean',
        'medianPalateScore' : 'median',
        'maxPalateScore' : 'max',
        'minPalateScore' : 'min',
        'palateScoreRange' : lambda x: max(x) - min(x)
    },
    'review_taste' : {
        'avgTasteScore' : 'mean',
        'medianTasteScore' : 'median',
        'maxTasteScore' : 'max',
        'minTasteScore' : 'min',
        'tasteScoreRange' : lambda x: max(x) - min(x)
    },
    'review_overall' : {
        'avgOverallScore' : 'mean',
        'medianOverallScore' : 'median',
        'maxOverallScore' : 'max',
        'minOverallScore' : 'min',
        'overallScoreRange' : lambda x: max(x) - min(x)
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[0],item[2]] for item in x]
    },
    'time_from_beginning' : {
        'avgTimestamp' : 'mean',
        'medianTimestamp' : 'median',
        'maxTimestamp' : 'max',
        'minTimestamp' : 'min',
        'TimestampRange' : lambda x: max(x) - min(x)
    },
    'weekday_central' : {
        'most_common_weekday' : lambda x: x.value_counts().index[0],
        'number_of_weekdays' : lambda x: x.nunique()
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month' : {
        'most_common_month' : lambda x: x.value_counts().index[0],
        'number_of_months' : lambda x: x.nunique()
    },
    'distance_from_july' : {
        'avgMonthsFromJuly' : 'mean',
        'medianMonthsFromJuly' : 'median',
        'maxMonthsFromJuly' : 'max',
        'minMonthsFromJuly' : 'min',
        'monthsFromJulyRange' : lambda x: max(x) - min(x)
    },
    'vader_neg' : {
        'avgVaderNeg' : 'mean',
        'medianVaderNeg' : 'median',
        'maxVaderNeg' : 'max',
        'minVaderNeg' : 'min',
        'vaderNegRange' : lambda x: max(x) - min(x)
    },
    'vader_neu' : {
        'avgVaderNeu' : 'mean',
        'medianVaderNeu' : 'median',
        'maxVaderNeu' : 'max',
        'minVaderNeu' : 'min',
        'vaderNeuRange' : lambda x: max(x) - min(x)
    },
    'vader_pos' : {
        'avgVaderPos' : 'mean',
        'medianVaderPos' : 'median',
        'maxVaderPos' : 'max',
        'minVaderPos' : 'min',
        'vaderPosRange' : lambda x: max(x) - min(x)
    }
}
aggByUser_df = SVD_set.groupby('reviewer_username').agg(aggregation)

/Users/conhealy/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [11]:
aggregation={
    'reviewer_username' : {
        'user_count' : 'count'
    },
    'abv' : {
        'avgABV' : 'mean',
        'medianABV' : 'median',
        'maxABV' : 'max',
        'minABV' : 'min',
        'ABVrange' : lambda x: max(x) - min(x)
    },
    'abv_listed' : {
        'num_abv_listed' : lambda x: int(sum(x)) 
    },
    'beer_style' : {
        'numBeerStyles' : lambda x: x.value_counts().index[0]
    },
    'review_appearance' : {
        'avgAppearanceScore' : 'mean',
        'medianAppearanceScore' : 'median',
        'maxAppearanceScore' : 'max',
        'minAppearanceScore' : 'min',
        'appearanceScoreRange' : lambda x: max(x) - min(x)
    },
    'review_aroma' : {
        'avgAromaScore' : 'mean',
        'medianAromaScore' : 'median',
        'maxAromaScore' : 'max',
        'minAromaScore' : 'min',
        'aromaScoreRange' : lambda x: max(x) - min(x)
    },
    'review_palate' : {
        'avgPalateScore' : 'mean',
        'medianPalateScore' : 'median',
        'maxPalateScore' : 'max',
        'minPalateScore' : 'min',
        'palateScoreRange' : lambda x: max(x) - min(x)
    },
    'review_taste' : {
        'avgTasteScore' : 'mean',
        'medianTasteScore' : 'median',
        'maxTasteScore' : 'max',
        'minTasteScore' : 'min',
        'tasteScoreRange' : lambda x: max(x) - min(x)
    },
    'review_overall' : {
        'avgOverallScore' : 'mean',
        'medianOverallScore' : 'median',
        'maxOverallScore' : 'max',
        'minOverallScore' : 'min',
        'overallScoreRange' : lambda x: max(x) - min(x)
    },
    'text_lookup' : {
        'text_list' : lambda x : [[item[1],item[2]] for item in x]
    },
    'time_from_beginning' : {
        'avgTimestamp' : 'mean',
        'medianTimestamp' : 'median',
        'maxTimestamp' : 'max',
        'minTimestamp' : 'min',
        'TimestampRange' : lambda x: max(x) - min(x)
    },
    'weekday_central' : {
        'most_common_weekday' : lambda x: x.value_counts().index[0],
        'number_of_weekdays' : lambda x: x.nunique()
    },
    'weekend_review' : {
        'num_weekend_reviews' : 'sum'
    },
    'month' : {
        'most_common_month' : lambda x: x.value_counts().index[0],
        'number_of_months' : lambda x: x.nunique()
    },
    'distance_from_july' : {
        'avgMonthsFromJuly' : 'mean',
        'medianMonthsFromJuly' : 'median',
        'maxMonthsFromJuly' : 'max',
        'minMonthsFromJuly' : 'min',
        'monthsFromJulyRange' : lambda x: max(x) - min(x)
    },
    'vader_neg' : {
        'avgVaderNeg' : 'mean',
        'medianVaderNeg' : 'median',
        'maxVaderNeg' : 'max',
        'minVaderNeg' : 'min',
        'vaderNegRange' : lambda x: max(x) - min(x)
    },
    'vader_neu' : {
        'avgVaderNeu' : 'mean',
        'medianVaderNeu' : 'median',
        'maxVaderNeu' : 'max',
        'minVaderNeu' : 'min',
        'vaderNeuRange' : lambda x: max(x) - min(x)
    },
    'vader_pos' : {
        'avgVaderPos' : 'mean',
        'medianVaderPos' : 'median',
        'maxVaderPos' : 'max',
        'minVaderPos' : 'min',
        'vaderPosRange' : lambda x: max(x) - min(x)
    }
}
aggByProduct_df = SVD_set.groupby('beerID').agg(aggregation)

/Users/conhealy/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [12]:
aggByUser_df.head()

vader_neg                                         \
                  avgVaderNeg medianVaderNeg maxVaderNeg minVaderNeg   
reviewer_username                                                      
 -BB-99              0.000000          0.000       0.000         0.0   
 007lund             0.067766          0.057       0.296         0.0   
 0087341             0.000000          0.000       0.000         0.0   
 00cobraR            0.051487          0.000       0.432         0.0   
 0110x011            0.033333          0.037       0.071         0.0   

                                           month                    \
                  vaderNegRange number_of_months most_common_month   
reviewer_username                                                    
 -BB-99                   0.000                1                11   
 007lund                  0.296               10                07   
 0087341                  0.000                1                03   
 00cobraR                 0.432               12                05   
 0110x011                 0.071                2                10   

                                                         text_lookup  \
                                                           text_list   
reviewer_username                                                      
 -BB-99            [[178, good california micro little hoppy me a...   
 007lund           [[1094, good stuff loads citrus incredible fin...   
 0087341           [[1266, best us beer enjoy new belgium 1554 be...   
 00cobraR          [[17557, pours nice hazy orange color small wh...   
 0110x011          [[89051, pours perfect pitch black thick foamy...   

                      abv_listed    brewerID      ...        \
                  num_abv_listed num_brewers      ...         
reviewer_username                                 ...         
 -BB-99                        1           1      ...         
 007lund                      46          32      ...         
 0087341                       1           1      ...         
 00cobraR                    522         201      ...         
 0110x011                      9           9      ...         

                  distance_from_july                                      \
                   avgMonthsFromJuly minMonthsFromJuly maxMonthsFromJuly   
reviewer_username                                                          
 -BB-99                     4.000000                 4                 4   
 007lund                    1.851064                 0                 6   
 0087341                    4.000000                 4                 4   
 00cobraR                   2.876866                 0                 6   
 0110x011                   3.444444                 3                 5   

                                          beer_style   vader_pos              \
                  medianMonthsFromJuly numBeerStyles minVaderPos avgVaderPos   
reviewer_username                                                              
 -BB-99                            4.0             1       0.097    0.097000   
 007lund                           1.0            21       0.057    0.251447   
 0087341                           4.0             1       0.420    0.420000   
 00cobraR                          3.0            67       0.000    0.204737   
 0110x011                          3.0             7       0.058    0.148667   

                                                            
                  vaderPosRange maxVaderPos medianVaderPos  
reviewer_username                                           
 -BB-99                   0.000       0.097          0.097  
 007lund                  0.586       0.643          0.234  
 0087341                  0.000       0.420          0.420  
 00cobraR                 0.515       0.515          0.203  
 0110x011                 0.166       0.224          0.134  

[5 rows x 65 columns]

In [13]:
aggByUser_df.columns = ['_'.join(col).strip() for col in aggByUser_df.columns.values] 

In [14]:
aggByUser_df.head()

,vader_neg_avgVaderNeg,vader_neg_medianVaderNeg,vader_neg_maxVaderNeg,vader_neg_minVaderNeg,vader_neg_vaderNegRange,month_number_of_months,month_most_common_month,text_lookup_text_list,abv_listed_num_abv_listed,brewerID_num_brewers,...,distance_from_july_avgMonthsFromJuly,distance_from_july_minMonthsFromJuly,distance_from_july_maxMonthsFromJuly,distance_from_july_medianMonthsFromJuly,beer_style_numBeerStyles,vader_pos_minVaderPos,vader_pos_avgVaderPos,vader_pos_vaderPosRange,vader_pos_maxVaderPos,vader_pos_medianVaderPos
reviewer_username,,,,,,,,,,,,,,,,,,,,,
-BB-99,0.000000,0.000,0.000,0.0,0.000,1,11,"[[178, good california micro little hoppy me a...",1,1,...,4.000000,4,4,4.0,1,0.097,0.097000,0.000,0.097,0.097
007lund,0.067766,0.057,0.296,0.0,0.296,10,07,"[[1094, good stuff loads citrus incredible fin...",46,32,...,1.851064,0,6,1.0,21,0.057,0.251447,0.586,0.643,0.234
0087341,0.000000,0.000,0.000,0.0,0.000,1,03,"[[1266, best us beer enjoy new belgium 1554 be...",1,1,...,4.000000,4,4,4.0,1,0.420,0.420000,0.000,0.420,0.420
00cobraR,0.051487,0.000,0.432,0.0,0.432,12,05,"[[17557, pours nice hazy orange color small wh...",522,201,...,2.876866,0,6,3.0,67,0.000,0.204737,0.515,0.515,0.203
0110x011,0.033333,0.037,0.071,0.0,0.071,2,10,"[[89051, pours perfect pitch black thick foamy...",9,9,...,3.444444,3,5,3.0,7,0.058,0.148667,0.166,0.224,0.134


In [15]:
aggByProduct_df.columns = ['_'.join(col).strip() for col in aggByProduct_df.columns.values] 

In [16]:
aggByProduct_df.head()

,vader_neg_avgVaderNeg,vader_neg_medianVaderNeg,vader_neg_maxVaderNeg,vader_neg_minVaderNeg,vader_neg_vaderNegRange,month_number_of_months,month_most_common_month,text_lookup_text_list,abv_listed_num_abv_listed,review_aroma_aromaScoreRange,...,abv_minABV,abv_ABVrange,abv_maxABV,abv_medianABV,distance_from_july_monthsFromJulyRange,distance_from_july_avgMonthsFromJuly,distance_from_july_minMonthsFromJuly,distance_from_july_maxMonthsFromJuly,distance_from_july_medianMonthsFromJuly,reviewer_username_user_count
beerID,,,,,,,,,,,,,,,,,,,,,
1,0.050641,0.0350,0.422,0.0,0.422,12,03,"[[ rubbishystuff, always kind liked beamish ch...",468,9,...,4.10,0.0,4.10,4.10,6,3.047009,0,6,3.0,468
2,0.065677,0.0430,0.463,0.0,0.463,12,03,"[[ clowe, amber hazy white foam medium bittern...",325,9,...,5.00,0.0,5.00,5.00,6,3.089231,0,6,3.0,325
3,0.047674,0.0345,0.287,0.0,0.287,12,03,"[[ joebudzjoe, good cheap high gravity beer cl...",172,7,...,8.00,0.0,8.00,8.00,6,2.982558,0,6,3.0,172
4,0.044106,0.0300,0.447,0.0,0.447,12,02,"[[ Walt, ive traded beer away six times never ...",123,7,...,6.40,0.0,6.40,6.40,6,3.422764,0,6,4.0,123
5,0.065977,0.0550,0.350,0.0,0.350,12,12,"[[ lithy, bottle thanks suttree amber colored ...",174,7,...,6.13,0.0,6.13,6.13,6,4.442529,0,6,5.0,174


### Aggregate level feature engineering

In [17]:
#aggByUser_df.columns

In [18]:
# brewer variety index - numBrewers/beerID_count
aggByUser_df['beer_variety_index'] = aggByUser_df['brewerID_num_brewers']/aggByUser_df['beerID_beerID_count']

# beer style variety index - numBeerStyles/beerID_count
aggByUser_df['beerStyle_variety_index'] = aggByUser_df['beer_style_numBeerStyles']/aggByUser_df['beerID_beerID_count']

# brewer variety index - numBrewers/beerID_count
aggByUser_df['brewer_variety_index'] = aggByUser_df['brewerID_num_brewers']/aggByUser_df['beerID_beerID_count']

# rate of reviewing - beerID_count/timeStampRange
aggByUser_df['rate_of_posting'] = aggByUser_df['beerID_beerID_count']/aggByUser_df['time_from_beginning_TimestampRange']

# weekday variety index - number_of_weekdays/beerID_count
aggByUser_df['weekday_variety_index'] = aggByUser_df['weekday_central_number_of_weekdays']/aggByUser_df['beerID_beerID_count']

# month variety index - number_of_months/beerID_count
aggByUser_df['month_variety_index'] = aggByUser_df['month_number_of_months']/aggByUser_df['beerID_beerID_count']

# weekend review ratio - num_weekend_reviews/beerID_count
aggByUser_df['weekend_review_ratio'] = aggByUser_df['weekend_review_num_weekend_reviews']/aggByUser_df['beerID_beerID_count']

In [19]:
aggByProduct_df.columns

Index(['vader_neg_avgVaderNeg', 'vader_neg_medianVaderNeg',
       'vader_neg_maxVaderNeg', 'vader_neg_minVaderNeg',
       'vader_neg_vaderNegRange', 'month_number_of_months',
       'month_most_common_month', 'text_lookup_text_list',
       'abv_listed_num_abv_listed', 'review_aroma_aromaScoreRange',
       'review_aroma_medianAromaScore', 'review_aroma_avgAromaScore',
       'review_aroma_minAromaScore', 'review_aroma_maxAromaScore',
       'weekday_central_most_common_weekday',
       'weekday_central_number_of_weekdays', 'review_overall_avgOverallScore',
       'review_overall_maxOverallScore', 'review_overall_medianOverallScore',
       'review_overall_overallScoreRange', 'review_overall_minOverallScore',
       'vader_neu_vaderNeuRange', 'vader_neu_avgVaderNeu',
       'vader_neu_medianVaderNeu', 'vader_neu_maxVaderNeu',
       'vader_neu_minVaderNeu', 'vader_pos_minVaderPos',
       'vader_pos_avgVaderPos', 'vader_pos_vaderPosRange',
       'vader_pos_maxVaderPos', 'vader_pos_m

In [20]:
# rate of reviewing - beerID_count/timeStampRange
aggByProduct_df['rate_of_posting'] = aggByProduct_df['reviewer_username_user_count']/aggByProduct_df['time_from_beginning_TimestampRange']

# weekend review ratio - num_weekend_reviews/beerID_count
aggByProduct_df['weekend_review_ratio'] = aggByProduct_df['weekend_review_num_weekend_reviews']/aggByProduct_df['reviewer_username_user_count']

## Join dataframes together to build final training set

In [21]:
train_set['beerID'].count()

1608290

In [22]:
aggByUser_df['reviewer_username']=aggByUser_df.index

In [23]:
aggByProduct_df['beerID']=aggByProduct_df.index

In [24]:
aggByUser_df.head()

,vader_neg_avgVaderNeg,vader_neg_medianVaderNeg,vader_neg_maxVaderNeg,vader_neg_minVaderNeg,vader_neg_vaderNegRange,month_number_of_months,month_most_common_month,text_lookup_text_list,abv_listed_num_abv_listed,brewerID_num_brewers,...,vader_pos_maxVaderPos,vader_pos_medianVaderPos,beer_variety_index,beerStyle_variety_index,brewer_variety_index,rate_of_posting,weekday_variety_index,month_variety_index,weekend_review_ratio,reviewer_username
reviewer_username,,,,,,,,,,,,,,,,,,,,,
-BB-99,0.000000,0.000,0.000,0.0,0.000,1,11,"[[178, good california micro little hoppy me a...",1,1,...,0.097,0.097,1.000000,1.000000,1.000000,inf,1.000000,1.000000,0.000000,-BB-99
007lund,0.067766,0.057,0.296,0.0,0.296,10,07,"[[1094, good stuff loads citrus incredible fin...",46,32,...,0.643,0.234,0.680851,0.446809,0.680851,0.000002,0.127660,0.212766,0.340426,007lund
0087341,0.000000,0.000,0.000,0.0,0.000,1,03,"[[1266, best us beer enjoy new belgium 1554 be...",1,1,...,0.420,0.420,1.000000,1.000000,1.000000,inf,1.000000,1.000000,0.000000,0087341
00cobraR,0.051487,0.000,0.432,0.0,0.432,12,05,"[[17557, pours nice hazy orange color small wh...",522,201,...,0.515,0.203,0.375000,0.125000,0.375000,0.000003,0.013060,0.022388,0.509328,00cobraR
0110x011,0.033333,0.037,0.071,0.0,0.071,2,10,"[[89051, pours perfect pitch black thick foamy...",9,9,...,0.224,0.134,1.000000,0.777778,1.000000,0.000002,0.222222,0.222222,1.000000,0110x011


In [25]:
df_partMerge = pd.merge(train_set, aggByUser_df, on='reviewer_username', how='inner', 
                        suffixes=('_orig', '_userAgg'))

In [26]:
df_partMerge['review_taste_tasteScoreRange'].count()

1568711

In [27]:
df_partMerge.columns.values

array(['abv', 'abv_listed', 'abv_lookup', 'beerID', 'beerStyle_lookup',
       'beer_name', 'beer_style', 'brewerID', 'brewerID_lookup',
       'day_of_year', 'distance_from_july', 'distance_from_july_lookup',
       'month', 'month_lookup', 'review_appearance',
       'review_appearance_lookup', 'review_aroma', 'review_aroma_lookup',
       'review_overall', 'review_overall_lookup', 'review_palate',
       'review_palate_lookup', 'review_taste', 'review_taste_lookup',
       'review_text', 'review_unix_time', 'reviewer_username',
       'sentiment_vader', 'testData', 'text_filtered', 'text_lookup',
       'text_simple', 'time_from_beginning', 'time_from_beginning_lookup',
       'vader_neg', 'vader_neg_lookup', 'vader_neu', 'vader_neu_lookup',
       'vader_pos', 'vader_pos_lookup', 'weekday_central',
       'weekday_central_lookup', 'weekend_review', 'vader_neg_avgVaderNeg',
       'vader_neg_medianVaderNeg', 'vader_neg_maxVaderNeg',
       'vader_neg_minVaderNeg', 'vader_neg_vaderNe

In [28]:
df_merged = pd.merge(df_partMerge, aggByProduct_df, left_on='beerID', right_on='beerID', how='inner'
         , suffixes=('', '_prodAgg'))

In [29]:
# number of rows increases because some beers aren't reviewed at all
df_merged['beerID'].count()

1488952

In [31]:
#dfm1, dfm2 = np.array_split(df_merged, 2)
#dfm1.to_pickle(path='./df_merged1.pkl')
#dfm2.to_pickle(path='./df_merged2.pkl')

dfm1 = pd.read_pickle('df_merged1.pkl')
dfm2 = pd.read_pickle('df_merged2.pkl')
df_merged=pd.concat([dfm1,dfm2])



In [4]:
df_merged.columns.values


array(['beerID', 'beer_name', 'abv', 'abv_listed', 'brewerID',
       'beer_style', 'reviewer_username', 'review_appearance',
       'review_aroma', 'review_palate', 'review_taste', 'review_overall',
       'review_text', 'review_unix_time', 'weekday_central', 'day_of_year',
       'month', 'distance_from_july', 'text_filtered', 'text_simple',
       'sentiment_vader', 'text_lookup', 'abv_lookup', 'brewerID_lookup',
       'beerStyle_lookup', 'review_appearance_lookup',
       'review_aroma_lookup', 'review_palate_lookup',
       'review_taste_lookup', 'review_overall_lookup', 'vader_neg',
       'vader_neu', 'vader_pos', 'time_from_beginning', 'weekend_review',
       'time_from_beginning_lookup', 'weekday_central_lookup',
       'month_lookup', 'distance_from_july_lookup', 'vader_neg_lookup',
       'vader_neu_lookup', 'vader_pos_lookup',
       'review_aroma_aromaScoreRange', 'review_aroma_maxAromaScore',
       'review_aroma_medianAromaScore', 'review_aroma_minAromaScore',
       '

In [41]:
df_merged.head()

,beerID,beer_name,abv,abv_listed,brewerID,beer_style,reviewer_username,review_appearance,review_aroma,review_palate,...,time_from_beginning_medianTimestamp_prodAgg,time_from_beginning_maxTimestamp_prodAgg,time_from_beginning_TimestampRange_prodAgg,vader_pos_avgVaderPos_prodAgg,vader_pos_vaderPosRange_prodAgg,vader_pos_maxVaderPos_prodAgg,vader_pos_minVaderPos_prodAgg,vader_pos_medianVaderPos_prodAgg,rate_of_posting_prodAgg,weekend_review_ratio_prodAgg
0,5868,St Peters Organic Ale,4.5,True,974,Bitter,abemorsten,3,6,4,...,216216000,350179200,312768000,0.151212,0.57,0.57,0.0,0.136,9.783610e-07,0.447712
1,5868,St Peters Organic Ale,4.5,True,974,Bitter,jazz88,3,6,3,...,216216000,350179200,312768000,0.151212,0.57,0.57,0.0,0.136,9.783610e-07,0.447712
2,5868,St Peters Organic Ale,4.5,True,974,Bitter,zach8270,3,6,3,...,216216000,350179200,312768000,0.151212,0.57,0.57,0.0,0.136,9.783610e-07,0.447712
3,5868,St Peters Organic Ale,4.5,True,974,Bitter,Angeloregon,4,7,3,...,216216000,350179200,312768000,0.151212,0.57,0.57,0.0,0.136,9.783610e-07,0.447712
4,5868,St Peters Organic Ale,4.5,True,974,Bitter,bierkoning,3,6,2,...,216216000,350179200,312768000,0.151212,0.57,0.57,0.0,0.136,9.783610e-07,0.447712


### Remove the example in question from each lookup list
This way the specific rreview features are not embedded in the aggregate values. 

In [32]:
df_merged[['beerID','beerID_beerID_count']].head()

,beerID,beerID_beerID_count
0,5868,155
1,5868,1054
2,5868,1240
3,5868,961
4,5868,2502


In [33]:
df_merged.iloc[[125872]]

,abv,abv_listed,abv_lookup,beerID,beerStyle_lookup,beer_name,beer_style,brewerID,brewerID_lookup,day_of_year,...,abv_maxABV_prodAgg,abv_medianABV_prodAgg,distance_from_july_monthsFromJulyRange_prodAgg,distance_from_july_avgMonthsFromJuly_prodAgg,distance_from_july_minMonthsFromJuly_prodAgg,distance_from_july_maxMonthsFromJuly_prodAgg,distance_from_july_medianMonthsFromJuly_prodAgg,reviewer_username_user_count,rate_of_posting_prodAgg,weekend_review_ratio_prodAgg
125872,10.5,True,"[28540, Pwn3d, 10.5]",28540,"[28540, Pwn3d, Belgian Strong Ale]",Unibroue Édition 2004,Belgian Strong Ale,180,"[28540, Pwn3d, 180]",361,...,10.5,10.5,6,2.75,0,6,3.0,252,0.000001,0.488095


In [34]:
df_merged.columns.values


array(['abv', 'abv_listed', 'abv_lookup', 'beerID', 'beerStyle_lookup',
       'beer_name', 'beer_style', 'brewerID', 'brewerID_lookup',
       'day_of_year', 'distance_from_july', 'distance_from_july_lookup',
       'month', 'month_lookup', 'review_appearance',
       'review_appearance_lookup', 'review_aroma', 'review_aroma_lookup',
       'review_overall', 'review_overall_lookup', 'review_palate',
       'review_palate_lookup', 'review_taste', 'review_taste_lookup',
       'review_text', 'review_unix_time', 'reviewer_username',
       'sentiment_vader', 'testData', 'text_filtered', 'text_lookup',
       'text_simple', 'time_from_beginning', 'time_from_beginning_lookup',
       'vader_neg', 'vader_neg_lookup', 'vader_neu', 'vader_neu_lookup',
       'vader_pos', 'vader_pos_lookup', 'weekday_central',
       'weekday_central_lookup', 'weekend_review', 'vader_neg_avgVaderNeg',
       'vader_neg_medianVaderNeg', 'vader_neg_maxVaderNeg',
       'vader_neg_minVaderNeg', 'vader_neg_vaderNe

In [35]:
# some lookup columns remained frm the dataframe from when I saved a pickle from a previous version of this notebook
# I'm removing them here. But if running all from scratch, this cell won't be necessary
df_merged=df_merged.drop(['abv_lookup',
       'brewerID_lookup', 'beerStyle_lookup', 'review_appearance_lookup',
       'review_aroma_lookup', 'review_palate_lookup',
       'review_taste_lookup', 'review_overall_lookup', 'time_from_beginning_lookup', 'weekday_central_lookup',
       'month_lookup', 'distance_from_july_lookup', 'vader_neg_lookup',
       'vader_neu_lookup', 'vader_pos_lookup', 'time_from_beginning_lookup', 'weekday_central_lookup', 'month_lookup', 'distance_from_july_lookup', 'vader_neg_lookup',
       'vader_neu_lookup', 'vader_pos_lookup'], axis=1)

In [36]:
# We woould not have review information in cases where we're trying to 
# recommend something that's not been reviewed before so we can drop all that stuff
df_merged=df_merged.drop(['review_appearance', 'review_aroma', 'review_palate', 'review_taste',
                'review_text', 'review_unix_time', 'weekday_central',
               'day_of_year', 'month', 'distance_from_july', 'text_filtered', 
               'text_simple', 'sentiment_vader', 'vader_neg', 'vader_neu',
               'vader_pos', 'time_from_beginning', 'weekend_review'], axis=1)



### Add collaborative  filtering scores to dataframe

In [37]:
df_merged.head()

,abv,abv_listed,beerID,beer_name,beer_style,brewerID,review_overall,reviewer_username,testData,text_lookup,...,abv_maxABV_prodAgg,abv_medianABV_prodAgg,distance_from_july_monthsFromJulyRange_prodAgg,distance_from_july_avgMonthsFromJuly_prodAgg,distance_from_july_minMonthsFromJuly_prodAgg,distance_from_july_maxMonthsFromJuly_prodAgg,distance_from_july_medianMonthsFromJuly_prodAgg,reviewer_username_user_count,rate_of_posting_prodAgg,weekend_review_ratio_prodAgg
0,4.5,True,5868,St Peters Organic Ale,Bitter,974,14,abemorsten,False,"[5868, abemorsten, bottlecardinal pours light...",...,4.5,4.5,6,2.866013,0,6,3.0,306,9.783610e-07,0.447712
1,4.5,True,5868,St Peters Organic Ale,Bitter,974,11,jazz88,False,"[5868, jazz88, bottle medium golden color tan...",...,4.5,4.5,6,2.866013,0,6,3.0,306,9.783610e-07,0.447712
2,4.5,True,5868,St Peters Organic Ale,Bitter,974,12,zach8270,False,"[5868, zach8270, bottle 500 ml slightly hazy...",...,4.5,4.5,6,2.866013,0,6,3.0,306,9.783610e-07,0.447712
3,4.5,True,5868,St Peters Organic Ale,Bitter,974,14,Angeloregon,False,"[5868, Angeloregon, updated aug 21 2008 malty...",...,4.5,4.5,6,2.866013,0,6,3.0,306,9.783610e-07,0.447712
4,4.5,True,5868,St Peters Organic Ale,Bitter,974,11,bierkoning,False,"[5868, bierkoning, orange color head light ho...",...,4.5,4.5,6,2.866013,0,6,3.0,306,9.783610e-07,0.447712


In [38]:
print(aromaSVD.predict('Angeloregon',''))

user: Angeloregon item:            r_ui = None   est = 6.28   {'was_impossible': False}


In [3]:
def aromaPredict(row):
    return aromaSVD.predict(row['reviewer_username'], row['beerID']).est

def appearanacePredict(row):
    return appearanaceSVD.predict(row['reviewer_username'], row['beerID']).est

def palatePredict(row):
    return palateSVD.predict(row['reviewer_username'], row['beerID']).est

def tastePredict(row):
    return tasteSVD.predict(row['reviewer_username'], row['beerID']).est

def overallPredict(row):
    return overallSVD.predict(row['reviewer_username'], row['beerID']).est

def benchmarkPredict(row):
    return standardSVD.predict(row['reviewer_username'], row['beerID']).est


In [6]:
df_merged['aromaSVD']=df_merged.apply(aromaPredict, axis=1)

In [41]:
df_merged['appearanaceSVD']=df_merged.apply(appearanacePredict, axis=1)

In [42]:
df_merged['palateSVD']=df_merged.apply(palatePredict, axis=1)

In [43]:
df_merged['tasteSVD']=df_merged.apply(tastePredict, axis=1)

In [10]:
df_merged['overallSVD']=df_merged.apply(overallPredict, axis=1)

In [5]:
#dff1, dff2 = np.array_split(df_merged, 2)
#dff1.to_pickle(path='./df_final1.pkl')
#dff2.to_pickle(path='./df_final2.pkl')

dff1 = pd.read_pickle('df_merged1.pkl')
dff2 = pd.read_pickle('df_merged2.pkl')
df_merged=pd.concat([dff1,dff2])

In [5]:
file_name = os.path.expanduser('~/benchmarkSVD_dump')
_, standardSVD = dump.load(file_name)

In [49]:
df_merged[['aromaSVD','appearanaceSVD','palateSVD','tasteSVD','overallSVD', 'review_overall','testData']]

,aromaSVD,appearanaceSVD,palateSVD,tasteSVD,overallSVD,review_overall,testData
0,5.798785,3.250575,3.344836,5.379055,13.530825,14,False
1,6.358591,2.913058,2.928688,5.262634,12.470336,11,False
2,5.981086,2.971897,3.070367,5.419182,13.411647,12,False
3,6.223292,3.548265,3.273529,5.545698,14.397025,14,False
4,6.302348,2.869591,2.486358,6.189397,11.354125,11,False
5,4.895616,3.087858,2.732270,6.400306,13.105869,12,False
6,4.372878,3.314183,2.982993,5.904672,12.598309,9,False
7,5.590132,3.161187,3.033197,6.056446,10.129574,11,False
8,3.953912,3.449355,3.297815,3.984381,9.518131,14,False
9,5.301535,2.982108,2.954263,6.554114,10.755706,10,False
